In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python import keras

# Load and Prepare Data
df = pd.read_csv("main_data.csv")
# Perform any necessary data cleaning or preprocessing here

# Select Features and Target
X = df[['vnindex_close','exchange_rate','interest_rate','inflation','sp500_close','gdp_growth']]  # Define your features
Y = df['vcb_close']  # Define your target

# Normalize Feature Data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# -- Define the sequence length --
seq_length = 5

X_seq, Y_seq = [], []

# -- Generate sequences --
for i in range(len(X) - seq_length):
    X_seq.append(X[i:i + seq_length])
    Y_seq.append(Y[i + seq_length])

X_seq = np.array(X_seq)
Y_seq = np.array(Y_seq)

# Split the Data into Train and Test Sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build and Compile the LSTM Model
model = keras.Sequential([
    keras.layers.LSTM(50, activation='tanh', return_sequences=True, input_shape=(seq_length, X_seq.shape[2])),
    keras.layers.LSTM(50, activation='tanh', return_sequences=True),
    keras.layers.LSTM(50, activation='tanh'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the Model
model.fit(X_train, Y_train, epochs=100, batch_size=64)

# Evaluate the Model
mse = model.evaluate(X_test, Y_test)
print(f"Mean Squared Error: {mse}")

# Save the trained model if needed
model.save("your_model_directory")


AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'